In [ ]:
import pandas as pd
import numpy as np
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 68.0 MB/s 


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Sheet2.xlsx to Sheet2.xlsx


In [ ]:
data1 = pd.read_excel('Sheet2.xlsx')

In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('no_print2.py','wb').write(src)

Saving no_print2.py to no_print2.py


6160

In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('clean_sword.py','wb').write(src)

Saving clean_sword.py to clean_sword.py


3805

In [ ]:
from clean_sword import i_stop_word
from no_print2 import extract_wq
data2 = extract_wq(data1)

삭제된 work: 5274,추가로 삭제된 qauli: 596, 총 삭제건수 : 5870


In [ ]:
data3 = i_stop_word(data2)

 관리            	18688
 개발            	13824
 기획            	9505
 운영            	9143
 분석            	7565
 채용            	6190
 사업            	6135
 기술            	5910
 영어            	5738
 대한            	5534
 설계            	4902
 전략            	4771
 작성            	4702
 수립            	4645
 능력            	4640
 대졸            	4550
 대리            	4427
 필수            	4411
 과장            	4302
 서비스           	4241
 시스템           	4190
 고객            	4095
 마케팅           	4086
 해외            	3972
 기업            	3932
 분야            	3854
 영업            	3838
 커뮤니케이션        	3702
 졸업            	3590
 수행            	3344
 프로젝트          	3318
 인사            	3204
 기타            	3174
 데이터           	3061
 활용            	2912
 차장            	2867
 기반            	2829
 역량            	2732
 이해            	2660
 구축            	2627
 검토            	2571
 협의            	2549
 제품            	2501
 사용            	2492
 개선            	2481
 대학교           	2447
 디자인           	2437
 산업        

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
# 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
text = [" ".join(noun) for noun in data3['full_work']]
#tfidf_vectorizer = TfidfVectorizer(max_features = 1231,min_df = 10, ngram_range=(1,5))
tfidf_vectorizer = TfidfVectorizer(min_df = 9, ngram_range=(1,5))
tfidf_vectorizer.fit(text)
vector1 = tfidf_vectorizer.transform(text).toarray()
# 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
text = [" ".join(noun) for noun in data3['full_quali']]
tfidf_vectorizer = TfidfVectorizer(min_df = 10, ngram_range=(1,5))
#tfidf_vectorizer = TfidfVectorizer(max_features = 1231,min_df = 10, ngram_range=(1,5))
tfidf_vectorizer.fit(text)
vector2 = tfidf_vectorizer.transform(text).toarray()
print(vector1.shape)
print(vector2.shape)

(12637, 14791)
(12637, 15747)


In [ ]:
shape_list = []
shape_list.append(vector1.shape[1])
shape_list.append(vector2.shape[1])
min_shape = np.min(shape_list)
print(min_shape)

14791


In [ ]:
#shape를 맞춘 값으로 다시 벡터화
text = [" ".join(noun) for noun in data3['full_work']]
tfidf_vectorizer = TfidfVectorizer(max_features = min_shape,min_df = 9, ngram_range=(1,5))
tfidf_vectorizer.fit(text)
vector1 = tfidf_vectorizer.transform(text).toarray()
text = [" ".join(noun) for noun in data3['full_quali']]
tfidf_vectorizer = TfidfVectorizer(max_features = min_shape,min_df = 10, ngram_range=(1,5))
tfidf_vectorizer.fit(text)
vector2 = tfidf_vectorizer.transform(text).toarray()
print(vector1.shape)
print(vector2.shape)

(12637, 14791)
(12637, 14791)


In [ ]:
len(data3)

1236

#elbow 기법을 통한 최적의 k값 도출, 군집분석 실시

In [ ]:
#성능 지표-> "각 데이터와 센트로이드 사이의 평균 제곱 거리로 도출한 모델의 이너셔(inertia)" 이니셔는 값이 작을수록 군집화가 잘된것이다.

In [ ]:
sse = []
for i in range(1,15):
    km = KMeans(n_clusters=i,init='k-means++', random_state=0)
    km.fit(vector1,vector2)
    sse.append(km.inertia_)

In [ ]:
elbow_list = []
for i in range(0,len(sse)-1):
  elbow_list.append(sse[i]-sse[i+1])
elbow_list2 = []
for i in range(0,len(elbow_list)-1):
  elbow_list2.append(elbow_list[i]/elbow_list[i+1])

In [ ]:
#vector3으로 확인해봐도(이니셔그래프) 클러스터가 3개일때 가장 낮은 이너셔(inertia_)를 보인다.->최적의 군집화 개수 (리스트는 2개,3개,4개~순서임)
print(elbow_list2)

[2.254793728048432,
 1.2229187394176984,
 0.8991835613725898,
 1.268113858294497,
 1.095638908610151,
 0.8783024283065128,
 1.1691368453161397,
 1.0540531062633751,
 1.1618095949385912,
 1.4775550016117391,
 0.5780707332087421,
 1.512075631243898]

In [ ]:
#elbow_list2의 인덱스를 통해 최적의 k값(best_k) 추출  ->elbow_list2의 값이 가장 클 때의 k값 (꺾임정도가 가장 큰 경우)
best_k = elbow_list2.index(np.max(elbow_list2))+2
print(best_k)

2


In [ ]:
init_centroid = 'k-means++'
km = KMeans(n_clusters=13, max_iter=50000,init=init_centroid,random_state=0)
cluster_label = km.fit_predict(vector1,vector2)

In [ ]:
init_centroid = 'k-means++'
km2 = KMeans(n_clusters=12, max_iter=50000,init=init_centroid,random_state=0)
cluster_label2 = km2.fit_predict(vector1,vector2)

In [ ]:
init_centroid = 'k-means++'
km = KMeans(n_clusters=best_k, max_iter=50000,init=init_centroid,random_state=0)
cluster_label = km.fit_predict(vector1,vector2)

In [ ]:
#각 클러스터별 군집화된 공고수
number0 = []
for i in cluster_label:
    if i == 0:
        number0.append(3)
print(len(number0))
number1 = []
for i in cluster_label:
    if i == 1:
        number1.append(3)
print(len(number1))
number2 = []
for i in cluster_label:
    if i == 2:
        number2.append(3)
print(len(number2))
number3 = []
for i in cluster_label:
    if i == 3:
        number3.append(3)
print(len(number3))
number4 = []
for i in cluster_label:
    if i == 4:
        number4.append(3)
print(len(number4))
number7 = []
for i in cluster_label:
    if i == 7:
        number7.append(3)
print(len(number7))
number8 = []
for i in cluster_label:
    if i == 8:
        number8.append(3)
print(len(number8))

4777
623
2163
240
1479
920
369


In [ ]:
#각 클러스터별 군집화된 공고수
number0 = []
for i in cluster_label2:
    if i == 0:
        number0.append(3)
print(len(number0))
number1 = []
for i in cluster_label2:
    if i == 1:
        number1.append(3)
print(len(number1))
number2 = []
for i in cluster_label2:
    if i == 2:
        number2.append(3)
print(len(number2))
number3 = []
for i in cluster_label2:
    if i == 3:
        number3.append(3)
print(len(number3))
number4 = []
for i in cluster_label2:
    if i == 4:
        number4.append(3)
print(len(number4))
number5 = []
for i in cluster_label2:
    if i == 5:
        number5.append(3)
print(len(number5))
number6 = []
for i in cluster_label2:
    if i == 6:
        number6.append(3)
print(len(number6))

NameError: ignored

In [ ]:
#군집화가 잘 이루어지면 컬럼 추가
data3['cluster_label'] = cluster_label

In [ ]:
cluster_10_words = []
def get_cluster_details(cluster_model, cluster_data, feature_names,
                       cluster_num, top_n_features=10):
    cluster_details = {}
    # 각 클러스터 레이블별 feature들의 center값들 내림차순으로 정렬 후의 인덱스를 반환
    center_feature_idx = cluster_model.cluster_centers_.argsort()[:,::-1]
    # 개별 클러스터 레이블별로 
    for cluster_num in range(cluster_num):
        # 개별 클러스터별 정보를 담을 empty dict할당
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # 각 feature별 center값들 정렬한 인덱스 중 상위 10개만 추출
        top_ftr_idx = center_feature_idx[cluster_num, :top_n_features]
        top_ftr = [feature_names[idx] for idx in top_ftr_idx]
        # top_ftr_idx를 활용해서 상위 10개 feature들의 center값들 반환
        # 반환하게 되면 array이기 떄문에 리스트로바꾸기
        top_ftr_val = cluster_model.cluster_centers_[cluster_num, top_ftr_idx].tolist()
        
        # cluster_details 딕셔너리에다가 개별 군집 정보 넣어주기
        cluster_details[cluster_num]['top_features'] = top_ftr
        cluster_details[cluster_num]['top_featrues_value'] = top_ftr_val
        # 해당 cluster_num으로 분류된 파일명(문서들) 넣어주기
        filenames = data3[data3['cluster_label']==cluster_num]['공고명']
        # filenames가 df으로 반환되기 떄문에 값들만 출력해서 array->list로 변환
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
    return cluster_details
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print(f"#####Cluster Num: {cluster_num}")
        print()
        print("상위 10개 feature단어들:\n", cluster_detail['top_features'])
        cluster_10_words.append(cluster_detail['top_features'])
        print()
        print(f"Cluster {cluster_num}으로 분류된 문서들:\n{cluster_detail['filenames'][:5]}")
        print('-'*20)

In [ ]:
#핵심단어 출력
feature_names = tfidf_vectorizer.get_feature_names()
eature_names = tfidf_vectorizer.get_feature_names()
cluster_details = get_cluster_details(cluster_model=km,
                                     cluster_data=data3,
                                     feature_names=feature_names,
                                     cluster_num=13,
                                     top_n_features=10)
print_cluster_details(cluster_details)

#####Cluster Num: 0

상위 10개 feature단어들:
 ['관리 채용 경영 관리', '유관 학과 졸업', '현재 소속사 경쟁', '최종 현재 희망', '이사 일정 합격자', '고급 인재 주요', '포털사이트 문의 실시간 확인', '납기', '대한 실무', '서비스 프로젝트']

Cluster 0으로 분류된 문서들:
['외국계 프리미엄 기업 인테리어 PM', '식음매장 인테리어 설계 및 시공(감리) (대기업계열 식품회사)', '인테리어담당(대기업계열 식품회사)', '[대기업 자회사] 전시 공간 기획 PM', '[국내 대기업] 조경 디자이너(플렌테리어) 경력 / 정규직 채용']
--------------------
#####Cluster Num: 1

상위 10개 feature단어들:
 ['효령로', '결정 현재 최종', '시뮬레이션', '정식 허가 등록 업체 시스템', '전문 아래 역량', '가지 모토로 차별 격조', '관리 채용 경영 관리', '더존 아이 큐브', '전화 이메일 연락 주시 상담', '제공']

Cluster 1으로 분류된 문서들:
['재무기획 부장급(팀장급)', '경영지원팀장 차(부)장급 경력사원 모집', '경영지원팀 임원-세무감사,외부감사 경력 필수/회계,세무 전반,인사 ,노무경영전반', '대형의류업체 베트남 현지법인 관리총괄 (부장/차장급) 정규직 채용', '경영관리팀 팀장(과장-부장)']
--------------------
#####Cluster Num: 2

상위 10개 feature단어들:
 ['능력 우수 영어', '모델링 데이터', '서비스 프로젝트', '제안', '사실', '신체 장애자 출시 관계', '이사 일정 합격자', '관리 채용 경영 관리', '세무 자격증 소지', '제어기']

Cluster 2으로 분류된 문서들:
['[대기업] 전시기획/공간기획 (스페이스/상설)', '분양전략/기획(임원/담당), 주거/호텔리조트 인테리어 디자인, 익스테리어 디자인(환경디자인), 상품개발, 광', '[스페이스마케팅] 대기

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#핵심단어 출력
feature_names = tfidf_vectorizer.get_feature_names()
eature_names = tfidf_vectorizer.get_feature_names()
cluster_details = get_cluster_details(cluster_model=km,
                                     cluster_data=data3,
                                     feature_names=feature_names,
                                     cluster_num=best_k,
                                     top_n_features=10)
print_cluster_details(cluster_details)

In [ ]:
cluster_10_words[1]

['효령로',
 '결정 현재 최종',
 '시뮬레이션',
 '정식 허가 등록 업체 시스템',
 '전문 아래 역량',
 '가지 모토로 차별 격조',
 '관리 채용 경영 관리',
 '더존 아이 큐브',
 '전화 이메일 연락 주시 상담',
 '제공']

#분류분석 시작 - 학습용 데이터셋 준비

In [ ]:
#full_noun을 통한 단어 벡터화 실시
reviews = list(data3['full_noun'])
sentences = []
for review in reviews:
    sentences.append(review)

In [ ]:
#기본 파라미터 설정
num_features = 300 #X.shape와 맞춰준다   
min_word_count = 40   
num_workers = 4       
context = 10          
downsampling = 1e-3 

In [ ]:
#gensim의 word2vec으로 sentences에 담겨있는 각 문서를 벡터화 (work와 quali가 합쳐진 full_noun)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
   level=logging.INFO)
   
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2022-06-26 13:59:41,406 : INFO : 'pattern' package not found; tag filters are not available for English
2022-06-26 13:59:41,416 : INFO : collecting all words and their counts
2022-06-26 13:59:41,417 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-06-26 13:59:41,516 : INFO : PROGRESS: at sentence #10000, processed 558181 words, keeping 7942 word types
2022-06-26 13:59:41,545 : INFO : collected 8788 word types from a corpus of 711441 raw words and 12637 sentences
2022-06-26 13:59:41,546 : INFO : Loading a fresh vocabulary
2022-06-26 13:59:41,555 : INFO : effective_min_count=40 retains 1774 unique words (20% of original 8788, drops 7014)
2022-06-26 13:59:41,559 : INFO : effective_min_count=40 leaves 657575 word corpus (92% of original 711441, drops 53866)
2022-06-26 13:59:41,567 : INFO : deleting the raw counts dictionary of 8788 items
2022-06-26 13:59:41,569 : INFO : sample=0.001 downsamples 72 most-common words
2022-06-26 13:59:41,572 : INFO : downsamplin

In [ ]:
# 모델 저장, word2vec.load()를 사용해서 추후에 다시 사용할 수 있다.
model_name = "w2v_300features_alljobs"
model.save(model_name)

2022-06-26 12:30:37,752 : INFO : saving Word2Vec object under w2v_300features_alljobs, separately None
2022-06-26 12:30:37,756 : INFO : not storing attribute vectors_norm
2022-06-26 12:30:37,758 : INFO : not storing attribute cum_table
2022-06-26 12:30:37,800 : INFO : saved w2v_300features_alljobs


In [ ]:
#하나의 리뷰에 대해 전체 단어의 평균값을 계산하는 함수
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [ ]:
#위의 함수를 이용해서 전체 리뷰에 대한 평균값을 계산하는 함수를 만들고, 실행
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [ ]:
#함수를 통해 학습 데이터셋 생성
test_data_vecs = get_dataset(sentences, model,300)
test_data_vecs.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


(12637, 300)

In [ ]:
# 출력변수 y 할당
X = test_data_vecs
y = data3.loc[:, 'cluster_label']
print(y.head())
print('X.shape:', X.shape, 'y.shape:', y.shape)

0     0
1    11
2     0
3     0
4     4
Name: cluster_label, dtype: int32
X.shape: (12637, 300) y.shape: (12637,)


In [ ]:
#필요모델 임포트
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score

In [ ]:
print(X.shape)
test_data_vecs = get_dataset(sentences, model,300)
print(test_data_vecs.shape)

(12637, 300)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


(12637, 300)


In [ ]:

#full_work를 벡터화시켜서(word2vec 활용) 학습데이터로 사용한다. -->full_noun으로 변경
X = test_data_vecs
y = np.array(data3['cluster_label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=28)

from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators=500)
lgbm.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='multiclass', eval_set=[(X_test,y_test)], verbose = True)

[1]	valid_0's multi_logloss: 1.65579
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 1.5088
[3]	valid_0's multi_logloss: 1.39737
[4]	valid_0's multi_logloss: 1.30626
[5]	valid_0's multi_logloss: 1.22986
[6]	valid_0's multi_logloss: 1.1652
[7]	valid_0's multi_logloss: 1.10664
[8]	valid_0's multi_logloss: 1.05442
[9]	valid_0's multi_logloss: 1.00763
[10]	valid_0's multi_logloss: 0.965278
[11]	valid_0's multi_logloss: 0.926735
[12]	valid_0's multi_logloss: 0.891179
[13]	valid_0's multi_logloss: 0.860514
[14]	valid_0's multi_logloss: 0.832636
[15]	valid_0's multi_logloss: 0.806872
[16]	valid_0's multi_logloss: 0.781948
[17]	valid_0's multi_logloss: 0.760669
[18]	valid_0's multi_logloss: 0.740097
[19]	valid_0's multi_logloss: 0.721286
[20]	valid_0's multi_logloss: 0.703843
[21]	valid_0's multi_logloss: 0.687695
[22]	valid_0's multi_logloss: 0.673167
[23]	valid_0's multi_logloss: 0.659383
[24]	valid_0's multi_logloss: 0.646278
[25]	valid_0's multi_

LGBMClassifier(n_estimators=500)

In [ ]:
#분류모델 정확도 출력
def get_clf_eval(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    print('정확도:\n', accuracy)
w_preds = lgbm.predict(X_test)
get_clf_eval(y_test, w_preds)

정확도:
 0.8623417721518988


In [ ]:
#분류모델 오차행렬 출력
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    print('오차행렬:\n', confusion)
get_clf_eval(y_test, w_preds)

오차행렬:
 [[876   8  36   0  18   0   1  17   4   2   0  11   3]
 [ 15 103   1   0   0   0   0   0   0   0   0   0   0]
 [ 38   0 372   0   4   0   0   2   0   0   0   0   0]
 [  4   0   0  37   2   0   0   0   0   0   0   0   0]
 [ 31   0   2   0 257   0   0   8   0   0   0   0   1]
 [  2   0   2   0   0  38   0   0   2   0   0   0   5]
 [ 10   0   1   0   1   0  26   2   0   0   0   0   0]
 [ 12   0   2   0  20   0   1 136   1   0   0   0   0]
 [ 15   0   4   0   2   3   0   2  47   2   0   0   8]
 [  4   0   1   0   0   0   0   0   0 105   0   0   0]
 [  1   0   1   0   0   0   0   0   0   0  81   0   0]
 [  3   0   5   0   1   0   0   0   0   0   0  55   0]
 [ 15   0   5   0   2   2   0   0   3   0   0   0  47]]


#단어분류, 중요 키워드 출력

In [ ]:
keyword_list = [['판매','영업','관리','기획','운영','분석','자동차','관리','판매',]]

In [ ]:
test_data_vecs2 = get_dataset(keyword_list, model,300)
test_data_vecs2.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


(1, 300)

In [ ]:
#분류모델을 통한 군집예측
cluster_pred = lgbm.predict(test_data_vecs2)
print(cluster_pred,cluster_10_words[cluster_pred[0]])

In [ ]:
stop_list=['위치','최고','개별','통보','부서','연봉','수준','면접','이력서','학력','무관','제출','직무','서류','직책','회사','담당','업무','포지션','자격','조건','근무','방법','접수','기간','전형','경력','우대','경험','이상','관련','자격','요건','사항','가능','직급','보유','지원']
def stopword(x):
  result = []
  for w in x:
      if w not in stop_list:
          result.append(w)
  return result

In [ ]:
print(3)

3


In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
nouns = okt.nouns('''기술 발달로 인해 세계화가 가속되고 있는 지금, 메타버스 산업과 사물 인터넷 등 지금까지는 보지 못했던 속도로 산업의 변화가 일어나고 있습니다. 특히 2020년부터 계속 심해지고 있는 코로나 상황 속에서 비대면 산업이 발달하고 실물의 구매 보다는 온라인을 통한 구매가 많아지고 있습니다.
이로 인해 많은 유통 구조가 변화하고 있으며 교육산업 또한 온라인으로 전환이 되고 있는 상황입니다.
아마 포스트 코로나 시대에는 이러한 변화의 기조가 유지될 것이라고 생각이 됩니다. 이러한 디지털 혁명의 산업 속에서 지속 가능한 산업에 대해 생각해봤을 때 쉽게 없어지지 않고 향후 발전적인 산업은 바로 농업과 물류, 건설과 같은 산업들이 있습니다.
또한 식품산업도 쉽게 도태되지 않을 산업들입니다. 저는 이러한 지속 가능한 산업과 관련된 회사에서 일하고 싶었습니다. 삼성물산은 화학, 철강, 에너지, 소재, 생활 산업 등을 원자재와 에너지 등 산업에 가장 기초적인 것들을 취급하기 때문에 향후 높은 발전가능성을 가지고 있다고 생각이 되어 지원하게 되었습니다.
긍정적인 성격과 사람들과 잘 어울리고 외국 문화와 문물에 누구보다도 빠른 적응이 저의 강점이라고 생각합니다. 주변에 외국인 친구들과 교류를 많이 하고 어렸을 때부터 해외여행을 자주 해서 현지 생활 적응도 빠릅니다. 또한 저는 영어와 일본어 그리고 아랍어를 할 수 있기 때문에 저의 외국어 역량을 글로벌 기업인 삼성 물산에서 마음 것 펼치고 싶습니다.
''')

In [ ]:
nouns1 = [nouns]

In [ ]:
nouns1

[['기술',
  '발달',
  '세계',
  '화가',
  '가속',
  '지금',
  '메타',
  '버스',
  '산업',
  '사물',
  '인터넷',
  '등',
  '지금',
  '보지',
  '속도',
  '산업',
  '변화',
  '일어나고',
  '계속',
  '코로나',
  '상황',
  '속',
  '비대',
  '산업',
  '발달',
  '실물',
  '구매',
  '온라인',
  '통한',
  '구매',
  '이로',
  '유통',
  '구조',
  '변화',
  '교육',
  '산업',
  '또한',
  '온라인',
  '전환',
  '상황',
  '아마',
  '포스트',
  '코로나',
  '시대',
  '변화',
  '기조',
  '유지',
  '것',
  '생각',
  '디지털',
  '혁명',
  '산업',
  '속',
  '지속',
  '산업',
  '대해',
  '생각',
  '때',
  '향후',
  '발전',
  '산업',
  '바로',
  '농업',
  '물류',
  '건설',
  '산업',
  '또한',
  '식품',
  '산업',
  '도태',
  '산업',
  '저',
  '지속',
  '산업',
  '관련',
  '회사',
  '일',
  '물산',
  '화학',
  '철강',
  '에너지',
  '소재',
  '생활',
  '산업',
  '등',
  '원자재',
  '에너지',
  '등',
  '산업',
  '가장',
  '기초',
  '것',
  '취급',
  '때문',
  '향후',
  '발전',
  '가능성',
  '가지',
  '생각',
  '지원',
  '긍정',
  '성격',
  '사람',
  '외국',
  '문화',
  '문물',
  '누구',
  '적응',
  '저',
  '강점',
  '생각',
  '주변',
  '외국인',
  '친구',
  '교류',
  '때',
  '해외여행',
  '자주',
  '현지',
  '생활',
  '적응도',
  '또한',
  '저',
  '영어',
  '일

In [ ]:
test_data_vecs2 = get_dataset(nouns1, model,300)
test_data_vecs2.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


(1, 300)

In [ ]:
#분류모델을 통한 군집예측
cluster_pred = lgbm.predict(test_data_vecs2)
print(cluster_pred,cluster_10_words[cluster_pred[0]])

[0] ['관리 채용 경영 관리', '유관 학과 졸업', '현재 소속사 경쟁', '최종 현재 희망', '이사 일정 합격자', '고급 인재 주요', '포털사이트 문의 실시간 확인', '납기', '대한 실무', '서비스 프로젝트']


In [ ]:
test_data_vecs2

array([[-0.12492308,  0.3152503 ,  0.23133056,  0.07055276,  0.0701743 ,
         0.04668871, -0.10104071,  0.05238223, -0.2360748 , -0.08505312,
         0.23731561, -0.21896501, -0.05282496, -0.03101308,  0.36309484,
         0.21204637,  0.14489782,  0.00613062, -0.09936274, -0.11858839,
        -0.1862546 ,  0.11813241,  0.02575823,  0.14861126, -0.03593323,
         0.25989458,  0.1133213 , -0.11410299, -0.11307477,  0.4374414 ,
         0.1799223 ,  0.08938187,  0.11923318,  0.19318639,  0.38022125,
         0.02406798,  0.12081317,  0.05685631,  0.12142418,  0.11818523,
         0.03335502,  0.09579631, -0.08705635, -0.15096633, -0.13825397,
         0.0763253 ,  0.15958863,  0.13450097, -0.02767249, -0.10130018,
        -0.03423523, -0.04827764, -0.03822123,  0.2576034 ,  0.02501093,
         0.20424442, -0.13556674, -0.3054963 , -0.05857945,  0.36026046,
         0.06316579,  0.0288805 , -0.09282766,  0.10041294, -0.13923045,
         0.11387811, -0.01643135, -0.11588186,  0.0

In [ ]:
abc11 = data3.groupby('cluster_label').get_group(0)

In [ ]:
abc11[60:80]

,산업명,공고명,경력,근무지역,직종_대분류,자격요건,업무내용,기업규모,등록일자,full_work,full_quali,full_noun,cluster_label
313,섬유·의복·모피,"용품 TD(사원-과장급)/(모자,가방패턴 경력 우대) - 패션 대기업",경력7년이상,서울 > 강남구,디자인,자격요건\nㆍ관련 경력 7년 이상 보유자\nㆍ가방패턴 및 제작경험 필수\nㆍ모자패턴...,ㆍ모자 가방의 용품 스펙 및 패턴 점검\nㆍ프리미엄 패션 브랜드의 용품 TD로 개발...,대기업,44686,"[모자, 가방, 용품, 스펙, 패턴, 점검, 프리미엄, 패션, 브랜드, 용품, 개발...","[가방, 패턴, 제작, 필수, 모자, 패턴, 봉제, 선호, 의류, 학과, 선호, 주...","[모자, 가방, 용품, 스펙, 패턴, 점검, 프리미엄, 패션, 브랜드, 용품, 개발...",0
314,섬유·의복·모피,"용품 스타일 디자이너 (선임~수석급)-국내 TOP 패션 대기업 [그룹사, 상장기업]",경력8년이상,서울 > 전지역,디자인,none,none,중소기업,44715,"[수주, 샘플, 드로잉, 도메인, 패션, 브랜드, 용품, 디자이너, 작성, 능력, ...","[출신, 지역, 혼인, 여부, 재산, 직계, 존비속, 형제자매, 직업]","[수주, 샘플, 드로잉, 도메인, 패션, 브랜드, 용품, 디자이너, 작성, 능력, ...",0
315,생활용품·화장품,컬러리스트 신입/경력 정규직 채용,신입/경력(연차무관),"경기 > 안산시 상록구, 안산시 단원구",디자인,"[자격요건]\n경력사항: 신입, 경력(1년이상)\n학력사항: 고등학교졸업 이상\n직...",[담당업무]\n컬러리스트(조색사)\n\n[근무부서 및 직급/직책]\n근무부서: 컬러...,소기업,44715,"[컬러, 리스트, 조색, 컬러, 개발, 사원]","[신입, 고등학교, 졸업, 기술, 컬러, 리스트, 자격증, 컬러, 리스트, 기사, ...","[컬러, 리스트, 조색, 컬러, 개발, 사원, 신입, 고등학교, 졸업, 기술, 컬러...",0
316,섬유·의복·모피,"의류 스타일 디자이너(선임급)-국내 TOP 패션 대기업 [그룹사, 상장기업]",경력4년이상,서울 > 전지역,디자인,none,none,중견기업,44715,"[수주, 샘플, 드로잉, 도메인, 패션, 브랜드, 스타일, 디자이너, 작성, 능력]","[출신, 지역, 혼인, 여부, 재산, 직계, 존비속, 형제자매, 직업]","[수주, 샘플, 드로잉, 도메인, 패션, 브랜드, 스타일, 디자이너, 작성, 능력,...",0
318,섬유·의복·모피,대형의류업체 본사 R&D센터 디자이너 (주임/사원급) 정규직 채용,경력1~5년,서울 > 전지역,디자인,none,none,중견기업,44714,"[스타일, 가먼트, 디자인, 기획, 스타, 일별, 일러스트, 스케치, 바이어, 미팅...","[수가, 필수, 이하, 정도, 주임, 사원, 영어, 능통, 해외, 유학, 체류, 참...","[스타일, 가먼트, 디자인, 기획, 스타, 일별, 일러스트, 스케치, 바이어, 미팅...",0
320,섬유·의복·모피,컬러리스트/(아웃도어/스포츠브랜드 경력 필수)/ (대리~과장급) - 패션 대기업,경력5년이상,서울 > 강남구,디자인,자격요건\nㆍ컬러리스트 경력 5년차 이상\nㆍ아웃도어/스포츠브랜드 경력 필수\n 기...,ㆍ컬러트렌드 분석을 통한 브랜드 라인별 시즌\n 컬러 및 컬러웨이 설정,대기업,44686,"[컬러, 트렌드, 분석, 통한, 브랜드, 라인, 시즌, 컬러, 컬러, 웨이, 설정]","[컬러, 리스트, 아웃도어, 스포츠, 브랜드, 필수, 기타, 고용, 형태, 정규직,...","[컬러, 트렌드, 분석, 통한, 브랜드, 라인, 시즌, 컬러, 컬러, 웨이, 설정,...",0
321,섬유·의복·모피,[외국계패션]소싱,경력5~10년,서울 > 강남구,디자인,ㆍ경력 : 5년 ~ 10년\nㆍ의류브랜드 생산 MD 경력\nㆍ의류학과 출신 선호\n...,"- 상품개발, 업체선정 및 발굴\n- 원가관리, 제품관리, 납기관리 등",중소기업,44658,"[상품, 개발, 업체, 선정, 발굴, 원가, 관리, 제품, 관리, 납기, 관리]","[의류, 브랜드, 생산, 의류, 학과, 출신, 선호, 출신, 선호]","[상품, 개발, 업체, 선정, 발굴, 원가, 관리, 제품, 관리, 납기, 관리, 의...",0
330,섬유·의복·모피,[패션기업-매출1조] 여성복 소재 디자이너,경력5~10년,서울 > 전지역,디자인,none,none,중견기업,44707,"[성복, 소재, 디자이너]","[성복, 브랜드, 소재, 디자인, 대리, 과장, 책임감, 주도, 성격, 중국, 공장...","[성복, 소재, 디자이너, 성복, 브랜드, 소재, 디자인, 대리, 과장, 책임감, ...",0
332,섬유·의복·모피,"중견기업 니트의류 OEM,ODM 수출디자인(R&D 디자이너) 임원급 구함(굿커리어)",경력10년이상,서울 > 전지역,디자인,"안녕하세요 \n굿커리어의 헤드헌터 유창덕입니다. \n저희는 금융,소비재팀과 반도체,...","[근무부서 및 직급/직책]\n직급/직책: 부장, 이사, 상무",중견기업,44705,"[부장, 이사, 상무]","[커리어, 헤드헌터, 유창, 저희, 금융, 소비재, 반도체, 전자, 컨설팅, 중공업...","[부장, 이사, 상무, 커리어, 헤드헌터, 유창, 저희, 금융, 소비재, 반도체, ...",0
333,섬유·의복·모피,"소재(우븐,데님,니트) 각 소싱 담당/(대리~과장급) - 패션 대기업",경력5년이상,서울 > 강남구,디자인,"자격요건\nㆍ대졸이상, 해당 업무 유경험자\nㆍ관련 경험 8년 이상 전문성 보유자/...","채용 포지션 및 담당직무\n1. (우븐, 데님) 소싱 담당 (과장급)\nㆍ글로벌 제...",대기업,44703,"[채용, 우븐, 데님, 소싱, 과장, 글로벌, 제품, 데님, 완제, 소재, 직소, ...","[대졸, 해당, 전문성, 대리, 영어, 유선, 무선, 영어, 중국어, 기타, 고용,...","[채용, 우븐, 데님, 소싱, 과장, 글로벌, 제품, 데님, 완제, 소재, 직소, ...",0
